In [1]:
import os
from osgeo import gdal
import rasterio
from rasterio import features
import numpy as np
import shapely

orthomosaic_path = "D:/FCAT/FCAT2APPK.tif"
output_dir = "D:/FCAT/background_crops" # folder where any outputs will be dumped

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [2]:
dataset = rasterio.open(orthomosaic_path)
band1 = dataset.read(1)
mask = band1 != 255

all_polygons = []
# for shape, value in features.shapes(mask.astype(np.int16), mask=(mask >0), transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0)):
for shape, value in features.shapes(mask.astype(np.int16), mask=(mask >0), transform=dataset.transform):
    all_polygons.append(shapely.geometry.shape(shape))

all_polygons = shapely.geometry.MultiPolygon(all_polygons)
if not all_polygons.is_valid:
    all_polygons = all_polygons.buffer(0)
    # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
    # need to keep it a Multi throughout
    if all_polygons.type == 'Polygon':
        all_polygons = shapely.geometry.MultiPolygon([all_polygons])

In [5]:
from shapely.geometry import mapping, Polygon
import fiona

# Define a polygon feature geometry with one attribute
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

# Write a new Shapefile
with fiona.open(output_dir + '/AOI_mask.shp', 'w', 'ESRI Shapefile', schema) as c:
    ## If there are multiple geometries, put the "for" loop here
    c.write({
        'geometry': mapping(all_polygons),
        'properties': {'id': 123},
    })